# 3W Real Dataset Cleaning & Preperation

### Imports & Configuration

In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
import sys
from pyspark.sql import SparkSession, Window, DataFrame
from pyspark.sql import functions as F
from pyspark.sql.functions import col, when, count, mean, isnan, expr, lit, countDistinct, round, lag, unix_timestamp, lead, explode, collect_list, struct, rand, row_number
from pyspark.sql.types import IntegerType, StringType, StructType, StructField, ArrayType, MapType
import pyspark.pandas as ps
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.functions import vector_to_array
from sklearn.model_selection import train_test_split
from functools import reduce
import json
from tqdm import tqdm

/opt/anaconda3/envs/pyspark-env/lib/python3.12/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [2]:
# Base directory paths
BASE_DIR = "../Cleaning & Preparation/"
STAGED_DATA_PATH = os.path.join(BASE_DIR, "Staged Cleaning Data")
RESULTS_PATH = os.path.join(BASE_DIR, "Results")
CLEANED_DATA_PATH = os.path.join(BASE_DIR, "Clean Data")
TRAIN_TEST_DATA_PATH = os.path.join(BASE_DIR, "Train Test Data")
SCALED_DATA_PATH = os.path.join(BASE_DIR, "Train Test (Scaled) Data")

In [3]:
### Initialize Spark Session
try:
    spark = SparkSession.builder \
        .appName("3W Real Dataset") \
        .master("local[*]") \
        .config("spark.driver.memory", "12g") \
        .config("spark.executor.memory", "6g") \
        .config("spark.sql.shuffle.partitions", "16") \
        .config("spark.default.parallelism", "16") \
        .config("spark.sql.parquet.filterPushdown", "true") \
        .config("spark.sql.parquet.enableVectorizedReader", "true") \
        .getOrCreate()
except Exception as e:
    print(f"Error initializing SparkSession: {e}")
    raise

24/11/27 08:02:39 WARN Utils: Your hostname, Ghazis-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 166.87.130.137 instead (on interface en0)
24/11/27 08:02:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/27 08:02:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
### Load Reference Table
REFERENCE_TABLE_PATH = "../EDA/Results/reference_table_real.csv"
reference_table_missing_values_real = spark.read.csv(REFERENCE_TABLE_PATH, header=True)
reference_table_missing_values_real = reference_table_missing_values_real.drop("Unique Values (Real)", "Missing Values (%) (Real)")
reference_table_missing_values_real.sort("Value Type").show(truncate=False, n=100)

+-------------+------------------------------------------------------------+--------------------+-----------+
|Tag          |Name                                                        |Unit                |Value Type |
+-------------+------------------------------------------------------------+--------------------+-----------+
|ESTADO-DHSV  |State of the DHSV (downhole safety valve)                   |[0, 0.5, 1]         |Categorical|
|ESTADO-M1    |State of the PMV (production master valve)                  |[0, 0.5, 1]         |Categorical|
|ESTADO-PXO   |State of the PXO (pig-crossover) valve                      |[0, 0.5, 1]         |Categorical|
|ESTADO-M2    |State of the AMV (annulus master valve)                     |[0, 0.5, 1]         |Categorical|
|ESTADO-SDV-GL|State of the gas lift SDV (shutdown valve)                  |[0, 0.5, 1]         |Categorical|
|ESTADO-SDV-P |State of the production SDV (shutdown valve)                |[0, 0.5, 1]         |Categorical|
|ESTADO-W1

In [5]:
### Define Features
feature_names = [
    row["Tag"] for row in reference_table_missing_values_real.filter(col("Value Type").isin(["Continuous", "Categorical"])).collect()
    if row["Tag"] != "DataType"
]
features_continuous = [
    row["Tag"] for row in reference_table_missing_values_real.filter(col("Value Type") == "Continuous").collect()
]
features_categorical = [
    row["Tag"] for row in reference_table_missing_values_real.filter(col("Value Type") == "Categorical").collect()
    if row["Tag"] != "DataType"
]

In [6]:
### Load Original 3W (Real) Dataset
DATA_PATH = "../Data/real_instances.parquet"
spark_df_real = spark.read.parquet(DATA_PATH)

In [7]:
### Fill Missing Values by Instance
def clean_missing_values(input_df: DataFrame, reference_table: DataFrame, output_path: str, incomplete_dirs: set = None):
    """
    Cleans missing values for each instance in the dataset by forward and backward filling.

    Parameters:
        input_df (DataFrame): Input Spark DataFrame containing the raw data.
        reference_table (DataFrame): Reference table specifying feature types.
        output_path (str): Directory path to save cleaned instances.
        incomplete_dirs (set): Optional. Set of directories that need reprocessing.

    Returns:
        None
    """
    reference_df = reference_table.select("Tag", "Value Type").distinct()
    relevant_columns = [
        row["Tag"] for row in reference_df.filter(col("Value Type").isin(["Continuous", "Categorical"])).collect()
        if row["Tag"] != "DataType"
    ]
    empty_columns = [row["Tag"] for row in reference_df.filter(col("Value Type") == "Empty").collect()]
    input_df = input_df.drop(*empty_columns)

    unique_instances = input_df.select("Instance", "label").distinct().collect()
    for row in tqdm(unique_instances, desc="Cleaning Instances"):
        instance = row["Instance"]
        label = row["label"]

        label_dir = os.path.join(output_path, f"Label_{label}")
        output_file = os.path.join(label_dir, f"Instance_{instance}.parquet")
        if incomplete_dirs:
            if output_file not in incomplete_dirs:
                continue

        try:
            instance_df = input_df.filter(col("Instance") == instance).orderBy("timestamp")
            total_rows = instance_df.count()
            for column in instance_df.columns:
                if instance_df.filter(col(column).isNull()).count() == total_rows:
                    instance_df = instance_df.drop(column)

            instance_pdf = instance_df.toPandas()
            for column in relevant_columns:
                if column in instance_pdf.columns:
                    instance_pdf[column] = instance_pdf[column].ffill().bfill()

            cleaned_sdf = spark.createDataFrame(instance_pdf)
            os.makedirs(label_dir, exist_ok=True)
            cleaned_sdf.write.mode("overwrite").parquet(output_file)
            print(f"Processed: Label - {label}, Instance - {instance}")

        except Exception as e:
            print(f"Error processing instance: Label - {label}, Instance - {instance}, Error: {type(e).__name__}: {e}")
            continue

    print(f"Cleaning process completed. Data exported to: {output_path}")

# Clean Missing Values
clean_missing_values(spark_df_real, reference_table_missing_values_real, os.path.join(STAGED_DATA_PATH, "Stage 1"))

Cleaning Instances:   0%|          | 1/1119 [00:06<2:05:48,  6.75s/it]          

Processed: Label - 2, Instance - 1095


Cleaning Instances:   0%|          | 2/1119 [00:13<2:01:31,  6.53s/it]

Processed: Label - 3, Instance - 1058


Cleaning Instances:   0%|          | 3/1119 [00:19<1:57:49,  6.33s/it]

...

Processed: Label - 4, Instance - 809


Cleaning Instances: 100%|█████████▉| 1118/1119 [1:52:52<00:04,  4.15s/it]

Processed: Label - 4, Instance - 833


Cleaning Instances: 100%|██████████| 1119/1119 [1:52:56<00:00,  6.06s/it]

Processed: Label - 4, Instance - 855
Cleaning process completed. Data exported to: ../Cleaning & Preparation/Staged Cleaning Data/Stage 1


In [8]:
### Transform Class Column to Binary Classification
def transform_class_column_in_cleaned_data(base_path, output_path):
    """
    Transforms the 'class' column into a binary classification column ('target').

    Parameters:
        base_path (str): Directory containing the cleaned instances.
        output_path (str): Directory path to save transformed instances.

    Returns:
        None
    """
    for label_dir in tqdm(os.listdir(base_path), desc="Transforming Class Column"):
        label_path = os.path.join(base_path, label_dir)
        if not os.path.isdir(label_path):
            continue

        for instance_dir in os.listdir(label_path):
            instance_path = os.path.join(label_path, instance_dir)
            if not instance_path.endswith(".parquet"):
                continue

            try:
                instance_df = spark.read.parquet(instance_path)
                if "class" not in instance_df.columns:
                    print(f"Skipping instance {instance_dir}: 'class' column not found.")
                    continue

                updated_df = instance_df.withColumn(
                    "target",
                    when(col("class").isin(None, 1, 2, 5, 6, 7, 8, 9), 0)
                    .when(col("class").isin(3, 4, 101, 102, 105, 106, 107, 108, 109), 1)
                    .otherwise(lit(0))
                )

                output_label_dir = os.path.join(output_path, label_dir)
                os.makedirs(output_label_dir, exist_ok=True)
                output_file = os.path.join(output_label_dir, instance_dir)
                updated_df.write.mode("overwrite").parquet(output_file)
                print(f"Processed: Label - {label_dir}, Instance - {instance_dir}")

            except Exception as e:
                print(f"Error processing {instance_path}: {e}")
                continue

    print(f"Processed data saved to: {output_path}")

# Transform Class Column
transform_class_column_in_cleaned_data(os.path.join(STAGED_DATA_PATH, "Stage 1"), os.path.join(STAGED_DATA_PATH, "Stage 2"))

Transforming Class Column:   0%|          | 0/10 [00:00<?, ?it/s]

Processed: Label - Label_9, Instance - Instance_11.parquet
Processed: Label - Label_9, Instance - Instance_5.parquet
Processed: Label - Label_9, Instance - Instance_18.parquet
Processed: Label - Label_9, Instance - Instance_45.parquet
Processed: Label - Label_9, Instance - Instance_55.parquet
Processed: Label - Label_9, Instance - Instance_27.parquet
Processed: Label - Label_9, Instance - Instance_37.parquet
Processed: Label - Label_9, Instance - Instance_19.parquet
Processed: Label - Label_9, Instance - Instance_54.parquet
Processed: Label - Label_9, Instance - Instance_44.parquet
Processed: Label - Label_9, Instance - Instance_36.parquet
Processed: Label - Label_9, Instance - Instance_26.parquet


Processed: Label - Label_9, Instance - Instance_10.parquet
Processed: Label - Label_9, Instance - Instance_4.parquet
Processed: Label - Label_9, Instance - Instance_24.parquet
Processed: Label - Label_9, Instance - Instance_34.parquet
Processed: Label - Label_9, Instance - Instance_46.parquet
Processed: Label - Label_9, Instance - Instance_56.parquet
Processed: Label - Label_9, Instance - Instance_6.parquet
Processed: Label - Label_9, Instance - Instance_12.parquet
Processed: Label - Label_9, Instance - Instance_7.parquet
Processed: Label - Label_9, Instance - Instance_13.parquet
Processed: Label - Label_9, Instance - Instance_35.parquet
Processed: Label - Label_9, Instance - Instance_25.parquet
Processed: Label - Label_9, Instance - Instance_47.parquet
Processed: Label - Label_9, Instance - Instance_20.parquet
Processed: Label - Label_9, Instance - Instance_30.parquet
Processed: Label - Label_9, Instance - Instance_42.parquet
Processed: Label - Label_9, Instance - Instance_52.parquet


Processed: Label - Label_9, Instance - Instance_2.parquet
Processed: Label - Label_9, Instance - Instance_16.parquet
Processed: Label - Label_9, Instance - Instance_28.parquet
Processed: Label - Label_9, Instance - Instance_38.parquet
Processed: Label - Label_9, Instance - Instance_3.parquet
Processed: Label - Label_9, Instance - Instance_17.parquet
Processed: Label - Label_9, Instance - Instance_31.parquet
Processed: Label - Label_9, Instance - Instance_21.parquet
Processed: Label - Label_9, Instance - Instance_53.parquet
Processed: Label - Label_9, Instance - Instance_43.parquet
Processed: Label - Label_9, Instance - Instance_48.parquet
Processed: Label - Label_9, Instance - Instance_15.parquet
Processed: Label - Label_9, Instance - Instance_1.parquet
Processed: Label - Label_9, Instance - Instance_41.parquet
Processed: Label - Label_9, Instance - Instance_51.parquet
Processed: Label - Label_9, Instance - Instance_23.parquet
Processed: Label - Label_9, Instance - Instance_33.parquet


Transforming Class Column:  10%|█         | 1/10 [00:37<05:39, 37.70s/it]

...

Transforming Class Column:  90%|█████████ | 9/10 [11:10<01:03, 63.92s/it]

Processed: Label - Label_5, Instance - Instance_1115.parquet
Processed: Label - Label_2, Instance - Instance_1095.parquet
Processed: Label - Label_2, Instance - Instance_1100.parquet


Processed: Label - Label_2, Instance - Instance_1101.parquet
Processed: Label - Label_2, Instance - Instance_1094.parquet
Processed: Label - Label_2, Instance - Instance_1103.parquet
Processed: Label - Label_2, Instance - Instance_1096.parquet
Processed: Label - Label_2, Instance - Instance_1086.parquet
Processed: Label - Label_2, Instance - Instance_1087.parquet
Processed: Label - Label_2, Instance - Instance_1097.parquet
Processed: Label - Label_2, Instance - Instance_1102.parquet
Processed: Label - Label_2, Instance - Instance_1107.parquet
Processed: Label - Label_2, Instance - Instance_1092.parquet
Processed: Label - Label_2, Instance - Instance_1093.parquet
Processed: Label - Label_2, Instance - Instance_1106.parquet
Processed: Label - Label_2, Instance - Instance_1091.parquet
Processed: Label - Label_2, Instance - Instance_1104.parquet
Processed: Label - Label_2, Instance - Instance_1088.parquet
Processed: Label - Label_2, Instance - Instance_1098.parquet
Processed: Label - Label

Transforming Class Column: 100%|██████████| 10/10 [11:24<00:00, 68.45s/it]

Processed: Label - Label_2, Instance - Instance_1090.parquet
Processed data saved to: ../Cleaning & Preparation/Staged Cleaning Data/Stage 2


In [9]:
### Combine Partitioned Data
def combine_partitioned_data_with_aligned_schema(base_path: str, original_df: DataFrame, output_file: str):
    """
    Combines partitioned Parquet files into a single dataset while aligning schemas.

    Parameters:
        base_path (str): Directory containing partitioned Parquet files.
        original_df (DataFrame): Original DataFrame to infer the schema.
        output_file (str): Path to save the combined dataset.

    Returns:
        DataFrame: Combined Spark DataFrame.
    """
    try:
        # Define the full schema (original schema + target column)
        full_schema = original_df.schema
        full_schema_fields = set(col.name for col in full_schema.fields)
        full_schema_fields.add("target")  # Ensure the 'target' column is included

        combined_df = None
        print(f"Traversing base path: {base_path}")
        
        for label_dir in os.listdir(base_path):
            label_path = os.path.join(base_path, label_dir)
            if not os.path.isdir(label_path):
                continue

            for instance_file in os.listdir(label_path):
                instance_path = os.path.join(label_path, instance_file)
                if not instance_path.endswith(".parquet"):
                    continue
                
                try:
                    instance_df = spark.read.parquet(instance_path)
                    print(f"Reading instance file: {instance_path}")
                    
                    # Align schema: Add missing columns with null values
                    instance_fields = set(instance_df.columns)
                    for col_name in full_schema_fields - instance_fields:
                        instance_df = instance_df.withColumn(col_name, lit(None))
                    
                    instance_df = instance_df.select(*full_schema_fields)

                    # Combine with the existing DataFrame
                    if combined_df is None:
                        combined_df = instance_df
                    else:
                        combined_df = combined_df.union(instance_df)

                except Exception as e:
                    print(f"Error reading {instance_path}: {type(e).__name__}: {e}")
                    continue

        # Drop VOID columns
        if combined_df is not None:
            void_columns = [col_name for col_name, dtype in combined_df.dtypes if dtype == "void"]
            if void_columns:
                print(f"Dropping VOID columns: {void_columns}")
                combined_df = combined_df.drop(*void_columns)


            # Save the combined dataset
            print(f"Saving combined dataset to: {output_file}")
            os.makedirs(os.path.dirname(output_file), exist_ok=True)
            combined_df.write.mode("overwrite").parquet(output_file)
            print(f"Combined dataset saved successfully at: {output_file}")

        else:
            print("No data to combine. Ensure the directory structure is correct and contains Parquet files.")

        return combined_df

    except Exception as e:
        print(f"Error combining partitioned data: {type(e).__name__}: {e}")
        return None

# Combine Partitioned Data
combined_df_real = combine_partitioned_data_with_aligned_schema(
    os.path.join(STAGED_DATA_PATH, "Stage 2"),
    spark_df_real,
    os.path.join(STAGED_DATA_PATH, "Stage 3", "real_instances_clean.parquet")
)

Traversing base path: ../Cleaning & Preparation/Staged Cleaning Data/Stage 2
Reading instance file: ../Cleaning & Preparation/Staged Cleaning Data/Stage 2/Label_9/Instance_11.parquet
Reading instance file: ../Cleaning & Preparation/Staged Cleaning Data/Stage 2/Label_9/Instance_5.parquet
Reading instance file: ../Cleaning & Preparation/Staged Cleaning Data/Stage 2/Label_9/Instance_18.parquet
Reading instance file: ../Cleaning & Preparation/Staged Cleaning Data/Stage 2/Label_9/Instance_45.parquet
Reading instance file: ../Cleaning & Preparation/Staged Cleaning Data/Stage 2/Label_9/Instance_55.parquet
Reading instance file: ../Cleaning & Preparation/Staged Cleaning Data/Stage 2/Label_9/Instance_27.parquet
Reading instance file: ../Cleaning & Preparation/Staged Cleaning Data/Stage 2/Label_9/Instance_37.parquet
Reading instance file: ../Cleaning & Preparation/Staged Cleaning Data/Stage 2/Label_9/Instance_19.parquet
Reading instance file: ../Cleaning & Preparation/Staged Cleaning Data/Stage 

24/11/27 02:44:35 WARN DAGScheduler: Broadcasting large task binary with size 19.6 MiB


Combined dataset saved successfully at: ../Cleaning & Preparation/Staged Cleaning Data/Stage 3/real_instances_clean.parquet


In [10]:
# Stage 3 Combined Clean by Instance Dataset Path
df_clean = spark.read.parquet(os.path.join(STAGED_DATA_PATH, "Stage 3", "real_instances_clean.parquet"))

24/11/27 02:58:24 WARN SharedInMemoryCache: Evicting cached table partition metadata from memory due to size constraints (spark.sql.hive.filesourcePartitionFileCacheSize = 262144000 bytes). This may impact query planning performance.


In [11]:
# Add Missingness Indicators
def create_missingness_columns(df, feature_names):
    """
    Adds missingness indicator columns for each feature in the dataset.

    Parameters:
        df (DataFrame): Input Spark DataFrame.
        feature_names (list): List of feature columns to process.

    Returns:
        DataFrame: Spark DataFrame with missingness indicator columns added.
    """
    for feature in feature_names:
        missing_indicator = f"{feature}_missing"
        df = df.withColumn(
            missing_indicator, when(col(feature).isNull(), lit(1)).otherwise(lit(0))
        )
    return df

df_with_missingness = create_missingness_columns(df_clean, feature_names)

In [12]:
# Impute Continuous Features
def impute_continuous_features(df, features_continuous, strategy="mean"):
    """
    Imputes missing values in continuous features using the specified strategy.

    Parameters:
        df (DataFrame): Input Spark DataFrame.
        features_continuous (list): List of continuous feature columns to impute.
        strategy (str): Imputation strategy ("mean" or "median").

    Returns:
        DataFrame: Spark DataFrame with imputed continuous features.
    """
    for feature in features_continuous:
        if strategy == "mean":
            imputed_value = df.select(mean(col(feature))).first()[0]
        elif strategy == "median":
            imputed_value = (
                df.approxQuantile(feature, [0.5], 0.01)[0]
                if df.select(col(feature)).dropna().count() > 0
                else None
            )
        else:
            raise ValueError("Unsupported strategy. Use 'mean' or 'median'.")

        if imputed_value is not None:
            df = df.withColumn(
                feature, when(col(feature).isNull(), lit(imputed_value)).otherwise(col(feature))
            )
    return df

df_imputed_continuous = impute_continuous_features(df_with_missingness, features_continuous, strategy="mean")

In [13]:
# Impute Categorical Features
def impute_categorical_features(df, features_categorical):
    """
    Imputes missing values in categorical features using the mode (most frequent value).

    Parameters:
        df (DataFrame): Input Spark DataFrame.
        features_categorical (list): List of categorical feature columns to impute.

    Returns:
        DataFrame: Spark DataFrame with imputed categorical features.
    """
    for feature in features_categorical:
        mode_value = (
            df.filter(col(feature).isNotNull())
              .groupBy(feature)
              .agg(count("*").alias("freq"))
              .orderBy(col("freq").desc())
              .select(feature)
              .first()[0]
        )
        if mode_value is not None:
            df = df.withColumn(
                feature, when(col(feature).isNull(), lit(mode_value)).otherwise(col(feature))
            )
    return df

df_clean = impute_categorical_features(df_imputed_continuous, features_categorical)

In [14]:
# Save Cleaned Dataset
cleaned_dataset_path = os.path.join(CLEANED_DATA_PATH, "cleaned_data.parquet")
df_clean.write.parquet(cleaned_dataset_path, mode="overwrite")

In [8]:
### Data Splitting into Train and Test Sets
def split_data(df, train_ratio=0.8):
    """
    Splits the DataFrame into train and test sets based on 'Instance', ensuring no overlap.
    """
    # Get distinct instances
    instances_df = df.select("Instance", "label").distinct()
    
    # Assign a random number to each instance
    instances_df = instances_df.withColumn("rand", rand(seed=42))
    
    # Handle labels with low number of instances
    window = Window.partitionBy("label").orderBy("rand")
    instances_df = instances_df.withColumn("row_number", row_number().over(window))
    instances_df = instances_df.withColumn("total_instances", F.count("*").over(Window.partitionBy("label")))
    
    # Define train and test indices
    instances_df = instances_df.withColumn(
        "split",
        when(
            (col("total_instances") > 4) & (col("row_number") <= col("total_instances") * train_ratio),
            "train"
        ).when(
            (col("total_instances") > 4),
            "test"
        ).when(
            (col("total_instances") <= 4) & (col("row_number") < col("total_instances")),
            "train"
        ).otherwise("test")
    )
    
    # Extract train and test instances
    train_instances = instances_df.filter(col("split") == "train").select("Instance")
    test_instances = instances_df.filter(col("split") == "test").select("Instance")
    
    # Join back to the main DataFrame
    train_df = df.join(train_instances, on="Instance", how="inner")
    test_df = df.join(test_instances, on="Instance", how="inner")
    
    return train_df, test_df

df_clean = spark.read.parquet(os.path.join(CLEANED_DATA_PATH, "cleaned_data.parquet"))
train_df, test_df = split_data(df_clean)

In [9]:
# Save Train and Test Data
train_data_path = os.path.join(TRAIN_TEST_DATA_PATH, "train_data.parquet")
test_data_path = os.path.join(TRAIN_TEST_DATA_PATH, "test_data.parquet")

train_df.write.parquet(train_data_path, mode="overwrite")
test_df.write.parquet(test_data_path, mode="overwrite")

24/11/27 08:03:42 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [10]:
### Verify Splits
def verify_splits(train_df, test_df):
    # Ensure no overlap in Instances
    train_instances = train_df.select("Instance").distinct()
    test_instances = test_df.select("Instance").distinct()
    overlapping_instances = train_instances.intersect(test_instances)
    overlap_count = overlapping_instances.count()
    if overlap_count > 0:
        print(f"Overlap detected in Instances: {overlap_count}")
    else:
        print("No overlap in Instances between train and test sets.")

    # Show counts per label
    print("Train set counts per label:")
    train_df.groupBy("label").agg(countDistinct("Instance").alias("instance_count")).show()
    print("Test set counts per label:")
    test_df.groupBy("label").agg(countDistinct("Instance").alias("instance_count")).show()

train_df = spark.read.parquet(train_data_path)
test_df = spark.read.parquet(test_data_path)
verify_splits(train_df, test_df)

No overlap in Instances between train and test sets.
Train set counts per label:
+-----+--------------+
|label|instance_count|
+-----+--------------+
|    9|            45|
|    3|            25|
|    4|           274|
|    6|             4|
|    2|            17|
|    7|            28|
|    8|            11|
|    5|             8|
|    0|           475|
|    1|             3|
+-----+--------------+

Test set counts per label:
+-----+--------------+
|label|instance_count|
+-----+--------------+
|    9|            12|
|    3|             7|
|    4|            69|
|    6|             2|
|    2|             5|
|    7|             8|
|    5|             3|
|    8|             3|
|    0|           119|
|    1|             1|
+-----+--------------+



In [11]:
### Apply Standard Scaler to Train and Test Data
def scale_features(train_df, test_df, feature_names):
    # Assemble features
    assembler = VectorAssembler(inputCols=feature_names, outputCol="features_vector", handleInvalid="skip")
    
    # Fit scaler on training data
    train_vectorized = assembler.transform(train_df)
    scaler = StandardScaler(inputCol="features_vector", outputCol="scaled_features", withStd=True, withMean=True)
    scaler_model = scaler.fit(train_vectorized)
    train_scaled = scaler_model.transform(train_vectorized)
    
    # Apply scaler to test data
    test_vectorized = assembler.transform(test_df)
    test_scaled = scaler_model.transform(test_vectorized)
    
    # Convert scaled features to array and update feature columns
    def extract_scaled_features(df):
        df = df.withColumn("scaled_features_array", vector_to_array(col("scaled_features")))
        for idx, feature in enumerate(feature_names):
            df = df.withColumn(feature, col("scaled_features_array")[idx])
        return df
    
    train_scaled = extract_scaled_features(train_scaled)
    test_scaled = extract_scaled_features(test_scaled)
    
    # Select relevant columns
    columns_to_keep = [col_name for col_name in df_clean.columns if col_name not in feature_names] + feature_names
    train_final_scaled = train_scaled.select(*columns_to_keep)
    test_final_scaled = test_scaled.select(*columns_to_keep)
    
    return train_final_scaled, test_final_scaled

train_final_scaled, test_final_scaled = scale_features(train_df, test_df, feature_names)

In [12]:
### Save Scaled Data
scaled_train_path = os.path.join(SCALED_DATA_PATH, "scaled_train_data.parquet")
scaled_test_path = os.path.join(SCALED_DATA_PATH, "scaled_test_data.parquet")

train_final_scaled.write.mode("overwrite").parquet(scaled_train_path)
test_final_scaled.write.mode("overwrite").parquet(scaled_test_path)

In [13]:
### End Spark Session
spark.stop()